In [ ]:
def get_penalty(team_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  shots = 0
  goal = 0
  for m in get_matches(team_wyid, season_wyid):
    events = get_events_per_match(m['matchId'])
    events_df = pd.DataFrame(events)
    events_df_type = pd.json_normalize(events_df['type'])
    events_df_team = pd.json_normalize(events_df['team'])
    events_df_team.columns = ['team_id', 'name', 'formation']
    events_df_team.drop(['name', 'formation'], axis = 1, inplace = True)
    events_df.drop(['type', 'team'], axis = 1, inplace = True)
    events_df = pd.concat((events_df, events_df_type, events_df_team), axis = 1)
    for i, e in events_df.iterrows():
      if e['primary'] == 'penalty' and e['team_id'] == team_wyid:
        e['shotType'] = 'PenaltyKick'
        shot.append(e)
  shots_df = pd.DataFrame(shot)
  for j, s in shots_df.iterrows():
    if s['shot']['isGoal'] == True:
      goal += 1
    else:
      shots += 1
  return shots_df, shots, goal

In [ ]:
def get_free_kick_shot(team_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  shots = 0
  goals = 0
  for m in get_matches(team_wyid, season_wyid):
    events = get_events_per_match(m['matchId'])
    events_df = pd.DataFrame(events)
    events_df_type = pd.json_normalize(events_df['type'])
    events_df_team = pd.json_normalize(events_df['team'])
    events_df_team.columns = ['team_id', 'name', 'formation']
    events_df_team.drop(['name', 'formation'], axis = 1, inplace = True)
    events_df.drop(['type', 'team'], axis = 1, inplace = True)
    events_df = pd.concat((events_df, events_df_type, events_df_team), axis = 1)
    for i, e in events_df.iterrows():
      if e['primary'] == 'free_kick' and 'free_kick_shot' in e['secondary'] and e['team_id'] == team_wyid:
        e['shotType'] = 'DirectFreeKick'
        shot.append(e)
  shots_df = pd.DataFrame(shot)
  for j, s in shots_df.iterrows():
    if s['shot']['isGoal'] == True:
      goals += 1
    else:
      shots += 1
  return shots_df, shots, goals

In [ ]:
def get_indirect_free_kick_shot(team_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  goal = 0
  shots = 0
  for m in get_matches(team_wyid, season_wyid):
    events = get_events_per_match(m['matchId'])
    events_df = pd.DataFrame(events)
    events_df_type = pd.json_normalize(events_df['type'])
    events_df_team = pd.json_normalize(events_df['team'])
    events_df_team.columns = ['team_id', 'name', 'formation']
    events_df_team.drop(['name', 'formation'], axis = 1, inplace = True)
    events_df.drop(['type', 'team'], axis = 1, inplace = True)
    events_df = pd.concat((events_df, events_df_type, events_df_team), axis = 1)
    events_df = events_df[events_df.primary != 'duel']
    events_df = events_df.reset_index()
    for i, e in events_df.iterrows():
      if e['primary'] == 'shot' and e ['team_id'] == team_wyid:
        if 'free_kick_cross' in events_df.loc[i-1, 'secondary']:
          e['shotType'] = 'IndirectFreeKickWithCross'
          shot.append(e)
        elif 'free_kick_cross' in events_df.loc[i-2, 'secondary'] and (events_df.loc[i-1,'primary'] == 'pass' and 'head_pass' in events_df.loc[i-1, 'secondary']):
          e['shotType'] = 'IndirectFreeKickWithCrossAndHeadBank'
          shot.append(e)
        elif 'free_kick_cross' in events_df.loc[i-2, 'secondary'] and (events_df.loc[i-1,'primary'] == 'pass' and 'head_pass' not in events_df.loc[i-1, 'secondary']):
          e['shotType'] = 'IndirectFreeKickWithCrossAndBank'
          shot.append(e)
        elif events_df.loc[i-2, 'primary'] == 'free_kick' and 'free_kick_shot' not in events_df.loc[i-2, 'secondary'] and get_distance(events_df.loc[i-2, 'location']['x'], events_df.loc[i-2, 'location']['y'], events_df.loc[i-2, 'pass']['endLocation']['x'], events_df.loc[i-2, 'pass']['endLocation']['y']) < 3:
          if events_df.loc[i-1, 'primary'] == 'pass' and 'cross' in events_df.loc[i-1, 'secondary']:
            e['shotType'] = 'IndirectFreeKickWithTouchAndCross'
            shot.append(e)
          elif events_df.loc[i-1, 'primary'] == 'touch':
            e['shotType'] = 'IndirectFreeKickWithTwoTouches'
            shot.append(e)
          elif events_df.loc[i-1, 'primary'] == 'pass':
            e['shotType'] = 'IndirectFreeKickWithTouchAndPass'
            shot.append(e)
        elif events_df.loc[i-1, 'primary'] == 'free_kick' and get_distance(events_df.loc[i-1, 'location']['x'], events_df.loc[i-1, 'location']['y'], events_df.loc[i-1, 'pass']['endLocation']['x'], events_df.loc[i-1, 'pass']['endLocation']['y']) < 3:
          e['shotType'] = 'IndirectFreeKickWithTouch'
          shot.append(e)

  shots_df = pd.DataFrame(shot)
  for j, s in shots_df.iterrows():
    if s['shot']['isGoal'] == True:
      goal += 1
    else:
      shots += 1
  return shots_df, shots, goal

In [ ]:
def get_throw_in_shot(team_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  goal = 0
  shots = 0
  for m in get_matches(team_wyid, season_wyid):
    events = get_events_per_match(m['matchId'])
    events_df = pd.DataFrame(events)
    events_df_type = pd.json_normalize(events_df['type'])
    events_df_team = pd.json_normalize(events_df['team'])
    events_df_team.columns = ['team_id', 'name', 'formation']
    events_df_team.drop(['name', 'formation'], axis = 1, inplace = True)
    events_df.drop(['type', 'team'], axis = 1, inplace = True)
    events_df = pd.concat((events_df, events_df_type, events_df_team), axis = 1)
    events_df = events_df[events_df.primary != 'duel']
    events_df = events_df.reset_index()
    for i, e in events_df.iterrows():
      if e['primary'] == 'shot' and e['team_id'] == team_wyid:
        first_throw_in = (events_df.loc[i::-1, 'primary'].eq('throw_in') & events_df.loc[i::-1, 'team_id'].eq(team_wyid)).idxmax() # controllare se non ci sono rimesse prima, se non ci sono rimesse prima prende lo stesso valore dell'indice del tiro
        if i- first_throw_in == 0:
          break
        elif i - first_throw_in <= 6:
          j = True
          for x in range(i, first_throw_in, -1):
              if events_df.loc[x, 'primary'] == 'free_kick' or events_df.loc[x, 'primary'] == 'corner': #events_df.loc[x, 'team_id'] != team_wyid or
                j = False
                break
          if j:
              e['shotType'] = 'shotAfterThrowIn'
              shot.append(e)




  shots_df = pd.DataFrame(shot)
  for x, s in shots_df.iterrows():
    if s['shot']['isGoal'] == True:
      goal += 1
    else:
      shots += 1
  return shots_df, shots, goal

In [ ]:
def get_ball_recovery(team_wyid, season_wyid = ''):
  shot = []
  goal = 0
  shots_df = pd.DataFrame(shot)
  shots = 0
  for m in get_matches(team_wyid, season_wyid):
    events = get_events_per_match(m['matchId'])
    events_df = pd.DataFrame(events)
    events_df_type = pd.json_normalize(events_df['type'])
    events_df_team = pd.json_normalize(events_df['team'])
    events_df_team.columns = ['team_id', 'name', 'formation']
    events_df_team.drop(['name', 'formation'], axis = 1, inplace = True)
    events_df.drop(['type', 'team'], axis = 1, inplace = True)
    events_df = pd.concat((events_df, events_df_type, events_df_team), axis = 1)
    events_df = events_df[events_df.primary != 'duel']
    events_df = events_df.reset_index()
    for i, e in events_df.iterrows():
      if e['primary'] == 'shot' and e ['team_id'] == team_wyid and 'shot_after_free_kick' not in e['secondary']:# and 'shot_after_throw_in' not in e['secondary']: #and 'shot_after_corner' not in e['secondary'] and
        last_opponent = (events_df.loc[i::-1, 'team_id'] != team_wyid).idxmax()
        if i- last_opponent <= 6:
          j = True
          for x in range(i, last_opponent, -1):
            if events_df.loc[x, 'primary'] == 'throw_in' or events_df.loc[x, 'primary'] == 'free_kick' or events_df.loc[x, 'primary'] == 'corner': #events_df.loc[x, 'team_id'] != team_wyid or
              j = False
            break

          if j:
            if events_df.loc[last_opponent+1, 'location']['x'] < 66:
              if events_df.loc[i, 'location']['x'] - events_df.loc[last_opponent+1, 'location']['x'] >25:
                if events_df.loc[last_opponent+1, 'location']['x'] < 38:
                  if events_df.loc[i, 'minute'] - events_df.loc[last_opponent+1, 'minute'] == 1:
                    if events_df.loc[i, 'second'] <= 14:
                      e['shotType'] = 'CounterAttackShot'
                      shot.append(e)
                  elif events_df.loc[i, 'second'] - events_df.loc[last_opponent+1, 'second'] <= 15 and(events_df.loc[i, 'minute'] - events_df.loc[last_opponent+1, 'minute'] == 0):
                    e['shotType'] = 'CounterAttackShot'
                    shot.append(e)
                elif events_df.loc[last_opponent+1, 'location']['x'] > 38 and events_df.loc[last_opponent+1, 'location']['x'] < 66:
                  if events_df.loc[i, 'minute'] - events_df.loc[last_opponent+1, 'minute'] == 1:
                    if events_df.loc[i, 'second'] <= 11:
                      e['shotType'] = 'CounterAttackShot'
                      shot.append(e)
                  elif events_df.loc[i, 'second'] - events_df.loc[last_opponent+1, 'second'] <= 12 and (events_df.loc[i, 'minute'] - events_df.loc[last_opponent+1, 'minute'] == 0):
                    e['shotType'] = 'CounterAttackShot'
                    shot.append(e)

            else:
              if i-last_opponent <= 3 and events_df.loc[last_opponent -1, 'team_id'] != team_wyid and 'shot_after_corner' not in e['secondary']:
                if 100 - events_df.loc[last_opponent+1, 'location']['x'] <25:
                  e['shotType'] = 'BallRecoveryShot'
                  shot.append(e)







  shots_df = pd.DataFrame(shot)
  for z, s in shots_df.iterrows():
    if s['shot']['isGoal'] == True:
      goal += 1
    else:
      shots += 1
  return shots_df, shots, goal

In [ ]:
def get_positional_attacks(team_wyid, season_wyid = ''):
  shot = []
  j = 0
  shots_df = pd.DataFrame(shot)
  goal = 0
  shots = 0
  for m in get_matches(team_wyid, season_wyid):
    events = get_events_per_match(m['matchId'])
    events_df = pd.DataFrame(events)
    events_df_type = pd.json_normalize(events_df['type'])
    events_df_team = pd.json_normalize(events_df['team'])
    events_df_team.columns = ['team_id', 'name', 'formation']
    events_df_team.drop(['name', 'formation'], axis = 1, inplace = True)
    events_df.drop(['type', 'team'], axis = 1, inplace = True)
    events_df = pd.concat((events_df, events_df_type, events_df_team), axis = 1)
    events_df = events_df[events_df.primary != 'duel']
    events_df = events_df.reset_index()
    for i, e in events_df.iterrows():
      j = 0
      if e['primary'] == 'shot' and e['team_id'] == team_wyid: #and 'shot_after_free_kick' not in e['secondary'] and 'shot_after_corner' not in e['secondary'] and 'shot_after_throw_in' not in e['secondary']
        last_opponent = (events_df.loc[i::-1, 'team_id'] != team_wyid).idxmax()
        if (events_df.loc[i-1, 'primary'] != 'free_kick' and events_df.loc[i-2, 'primary'] != 'free_kick') and (events_df.loc[i-1, 'primary'] != 'corner' and events_df.loc[i-2, 'primary'] != 'corner'):
          for x in range(i, last_opponent, -1):
            if (events_df.loc[x, 'team_id'] == team_wyid and \
               (events_df.loc[x, 'primary'] != 'throw_in' and events_df.loc[x, 'primary'] != 'free_kick' and events_df.loc[x, 'primary'] != 'corner')):
                j +=1
                if j>6:
                  e['shotType'] = 'positionalAttack'
                  shot.append(e)
                  break
            else:
              j = 0
              break


  shots_df = pd.DataFrame(shot)
  for x, s in shots_df.iterrows():
    if s['shot']['isGoal'] == True:
      goal += 1
    else:
      shots += 1

  return shots_df, shots, goal

In [ ]:
def get_direct_corner_kick(team_wyid, season_wyid = ''):
  shot = []
  shots = 0
  shots_df = pd.DataFrame(shot)
  goal = 0
  for m in get_matches(team_wyid, season_wyid):
    events = get_events_per_match(m['matchId'])
    events_df = pd.DataFrame(events)
    events_df_type = pd.json_normalize(events_df['type'])
    events_df_team = pd.json_normalize(events_df['team'])
    events_df_team.columns = ['team_id', 'name', 'formation']
    events_df_team.drop(['name', 'formation'], axis = 1, inplace = True)
    events_df.drop(['type', 'team'], axis = 1, inplace = True)
    events_df = pd.concat((events_df, events_df_type, events_df_team), axis = 1)
    for i, e in events_df.iterrows():
      if e['primary'] == 'corner' and e['team_id'] == team_wyid and 'shot' in e['secondary']:
        e['shotType'] = 'DirectCornerKick'
        shot.append(e)

    shots_df = pd.DataFrame(shot)
    for j, s in shots_df.iterrows():
      if 'goal' in s['secondary']:
        goal += 1
      else:
        shots += 1
  return shots_df, shots, goal

In [ ]:
def get_corner_kick_schema(team_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  shots = 0
  goal = 0
  for m in get_matches(team_wyid, season_wyid):
    events = get_events_per_match(m['matchId'])
    events_df = pd.DataFrame(events)
    events_df_type = pd.json_normalize(events_df['type'])
    events_df_team = pd.json_normalize(events_df['team'])
    events_df_team.columns = ['team_id', 'name', 'formation']
    events_df_team.drop(['name', 'formation'], axis = 1, inplace = True)
    events_df.drop(['type', 'team'], axis = 1, inplace = True)
    events_df = pd.concat((events_df, events_df_type, events_df_team), axis = 1)
    events_df = events_df[events_df.primary != 'duel']
    events_df = events_df.reset_index()
    for i, e in events_df.iterrows():
      if e['primary'] == 'shot' and e ['team_id'] == team_wyid:
        if 'shot_assist' in events_df.loc[i-1, 'secondary'] and events_df.loc[i-1, 'primary'] == 'corner':
          e['shotType'] = 'IndirectCornerKickWithCross'
          shot.append(e)
        elif 'cross' in events_df.loc[i-2, 'secondary'] and events_df.loc[i-2, 'primary'] == 'corner' and (events_df.loc[i-1,'primary'] == 'pass' and 'head_pass' in events_df.loc[i-1, 'secondary']):
          e['shotType'] = 'IndirectCornerKickWithCrossAndHeadBank'
          shot.append(e)
        elif 'cross' in events_df.loc[i-2, 'secondary'] and events_df.loc[i-2, 'primary'] == 'corner' and (events_df.loc[i-1,'primary'] == 'pass' and 'head_pass' not in events_df.loc[i-1, 'secondary']):
          e['shotType'] = 'IndirectCornerKickWithCrossAndBank'
          shot.append(e)
        elif events_df.loc[i-2, 'primary'] == 'corner' and get_distance(events_df.loc[i-2, 'location']['x'], events_df.loc[i-2, 'location']['y'], events_df.loc[i-2, 'pass']['endLocation']['x'], events_df.loc[i-2, 'pass']['endLocation']['y']) < 3:
          if events_df.loc[i-1, 'primary'] == 'pass' and 'cross' in events_df.loc[i-1, 'secondary']:
            e['shotType'] = 'IndirectCornerKickWithTouchAndCross'
            shot.append(e)
          elif events_df.loc[i-1, 'primary'] == 'touch':
            e['shotType'] = 'IndirectCornerKickWithTwoTouches'
            shot.append(e)
          elif events_df.loc[i-1, 'primary'] == 'pass':
            e['shotType'] = 'IndirectCornerKickWithTouchAndPass'
            shot.append(e)
        elif events_df.loc[i-1, 'primary'] == 'corner' and get_distance(events_df.loc[i-1, 'location']['x'], events_df.loc[i-1, 'location']['y'], events_df.loc[i-1, 'pass']['endLocation']['x'], events_df.loc[i-1, 'pass']['endLocation']['y']) < 3:
          e['shotType'] = 'IndirectCornerKickWithTouch'
          shot.append(e)

  shots_df = pd.DataFrame(shot)
  for j, s in shots_df.iterrows():
    if s['shot']['isGoal'] == True:
      goal += 1
    else:
      shots += 1
  return shots_df, shots, goal

In [ ]:
def get_gks_punch(team_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  shots = 0
  goals = 0
  for m in get_matches(team_wyid, season_wyid):
    events = get_events_per_match(m['matchId'])
    events_df = pd.DataFrame(events)
    events_df_type = pd.json_normalize(events_df['type'])
    events_df_team = pd.json_normalize(events_df['team'])
    events_df_team.columns = ['team_id', 'name', 'formation']
    events_df_team.drop(['name', 'formation'], axis = 1, inplace = True)
    events_df.drop(['type', 'team'], axis = 1, inplace = True)
    events_df = pd.concat((events_df, events_df_type, events_df_team), axis = 1)
    events_df = events_df[events_df.primary != 'duel']
    events_df = events_df.reset_index()
    for i, e in events_df.iterrows():
      if e['primary'] == 'shot' and e['team_id'] == team_wyid:
        if events_df.loc[i-1, 'primary'] == 'shot_against' and ('save' in events_df.loc[i-1, 'secondary'] or 'reflexes_save' in events_df.loc[i-1, 'secondary']):# and events_df.loc[i-1, 'player']['position'] == 'GK':
          e['shotType'] = 'shotAfterPunch'
          shot.append(e)
  shots_df = pd.DataFrame(shot)
  for j, s in shots_df.iterrows():
    if 'goal' in e['secondary']:
      goals += 1
    else:
      shots += 1

  return shots_df, shots, goals

In [ ]:
def get_own_goal(team_wyid, season_wyid = ''):
  shot = []
  own = 0
  shots_df = pd.DataFrame(shot)
  for m in get_matches(team_wyid, season_wyid):
    events = get_events_per_match(m['matchId'])
    events_df = pd.DataFrame(events)
    events_df_type = pd.json_normalize(events_df['type'])
    events_df_team = pd.json_normalize(events_df['team'])
    events_df_team.columns = ['team_id', 'name', 'formation']
    events_df_team.drop(['name', 'formation'], axis = 1, inplace = True)
    events_df.drop(['type', 'team'], axis = 1, inplace = True)
    events_df = pd.concat((events_df, events_df_type, events_df_team), axis = 1)
    for i, e in events_df.iterrows():
      if e['primary'] == 'own_goal' and e ['team_id'] == team_wyid:
         e['shotType'] = 'OwnGoal'
         shot.append(e)


  shots_df = pd.DataFrame(shot)
  for j,s in shots_df.iterrows():
    own += 1
  return shots_df, own